<a href="https://colab.research.google.com/github/tjwei/taishin20190827/blob/master/CF/svd_txn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 用 KNN 的方式做推薦

台新分析開發環境案客戶資料
https://drive.google.com/drive/u/2/folders/1AAXWPcB9xiDe_qX87ce5XL-utkASWxRe

請先連到這個分享資料夾，並且按下右鍵，加入你的 Google 雲端硬碟


In [1]:
# 這個是授權存取你的 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 確認真的連上了資料
!ls drive/My\ Drive/台新分析開發環境案客戶資料

'Copy of 0606_口述台新資料欄位.m4a'
'Copy of 0610_嘉駿寄來的欄位內容.png'
'Copy of 0621_data_original.zip'
'Copy of profile.csv'
'Copy of profile.gsheet'
'Copy of result_y1.csv'
'Copy of result_y1.gsheet'
'Copy of result_y2.csv'
'Copy of sr_1.csv'
'Copy of sr_2.csv'
'Copy of sr_3.csv'
'Copy of sr_4.csv'
'Copy of status.csv'
'Copy of visa-merchant-data-standards-manual_用來對照sr_1.csv中的mcc欄位.pdf'
'Copy of zip 對照表.xlsx'
'Copy of 檔案欄位說明_MOOCS.XLSX'


In [0]:
# 正式開始
import pandas as pd
import os.path
import numpy as np

In [0]:
DATA_DIR='drive/My Drive/台新分析開發環境案客戶資料'
sr_1_fn = os.path.join(DATA_DIR, 'Copy of sr_1.csv')
with open(sr_1_fn, encoding='cp950', errors='replace') as f:
    df = pd.read_csv(f, nrows=10000000, usecols=['CUST_NO', 'TXN_DESC'])

In [5]:
df.head()

,CUST_NO,TXN_DESC
0,564755,全聯台中四平分公司 TAICHU/TW
1,564755,全聯台中四平分公司 TAICHU/TW
2,6185152,健身工廠九如廠 Kaohsi/TW
3,6185152,富邦ｍｏｍｏ－ＥＣ TAIPEI/TW
4,5318034,國泰人壽保費部 第 12/12 期 /TW


In [6]:
df.TXN_DESC.nunique()

1159141

In [7]:
TXN = df.TXN_DESC.str.extract(r'([^\s０－９（５１２）]+)')
df['TXN_DESC'] = TXN
df.TXN_DESC.nunique()

117991

In [8]:
# 看一下使用頻率
txn_counts = df.TXN_DESC.value_counts()
txn_counts.quantile(np.arange(.5,1,.01))

0.50      2.0
0.51      2.0
0.52      2.0
0.53      3.0
0.54      3.0
0.55      3.0
0.56      3.0
0.57      3.0
0.58      3.0
0.59      3.0
0.60      4.0
0.61      4.0
0.62      4.0
0.63      4.0
0.64      4.0
0.65      5.0
0.66      5.0
0.67      5.0
0.68      5.0
0.69      6.0
0.70      6.0
0.71      6.0
0.72      7.0
0.73      7.0
0.74      7.0
0.75      8.0
0.76      9.0
0.77      9.0
0.78     10.0
0.79     11.0
0.80     12.0
0.81     13.0
0.82     14.0
0.83     15.0
0.84     17.0
0.85     18.0
0.86     20.0
0.87     23.0
0.88     26.0
0.89     29.0
0.90     34.0
0.91     40.0
0.92     48.0
0.93     59.0
0.94     75.0
0.95    100.0
0.96    144.0
0.97    224.0
0.98    396.0
0.99    868.3
Name: TXN_DESC, dtype: float64

In [9]:
# 只留下交易一定次數以上的商店
good_txn = txn_counts[txn_counts>100]
#df = df.loc[df.TXN_DESC.isin(good_txn.index)]
df.drop(df.loc[~df.TXN_DESC.isin(good_txn.index)].index, inplace=True)
df.TXN_DESC.nunique()

5898

In [10]:
# 看看用戶交易次數
cust_counts = df.CUST_NO.value_counts()
cust_counts.quantile(np.arange(0,1,.01))

0.00      1.0
0.01      1.0
0.02      1.0
0.03      1.0
0.04      2.0
0.05      2.0
0.06      2.0
0.07      3.0
0.08      3.0
0.09      3.0
0.10      4.0
0.11      4.0
0.12      5.0
0.13      5.0
0.14      5.0
0.15      6.0
0.16      6.0
0.17      6.0
0.18      7.0
0.19      7.0
0.20      7.0
0.21      8.0
0.22      8.0
0.23      8.0
0.24      9.0
0.25      9.0
0.26     10.0
0.27     10.0
0.28     11.0
0.29     11.0
        ...  
0.70     42.0
0.71     44.0
0.72     45.0
0.73     46.0
0.74     48.0
0.75     49.0
0.76     51.0
0.77     53.0
0.78     55.0
0.79     56.0
0.80     58.0
0.81     61.0
0.82     63.0
0.83     65.0
0.84     68.0
0.85     70.0
0.86     73.0
0.87     76.0
0.88     80.0
0.89     84.0
0.90     88.0
0.91     93.0
0.92     98.0
0.93    104.0
0.94    111.0
0.95    120.0
0.96    131.0
0.97    145.0
0.98    166.0
0.99    206.0
Name: CUST_NO, Length: 100, dtype: float64

In [11]:
good_cust = cust_counts[cust_counts>120]
#df = df.loc[df.CUST_NO.isin(good_cust.index)]
df.drop(df.loc[~df.CUST_NO.isin(good_cust.index)].index, inplace=True)
print(df.TXN_DESC.nunique())
print(df.CUST_NO.nunique())

5869
11673


In [12]:
# pivot dataframe, 變成我們要的矩陣
df = df.pivot_table(index='TXN_DESC', columns='CUST_NO',  aggfunc=len, fill_value=0 )
df

CUST_NO,37917,41741,44429,44792,45159,48042,48141,48428,49923,50985,55411,58179,59378,60109,60131,60437,61599,61714,62288,65300,66057,66593,67047,67296,67362,69724,70605,71301,73073,74410,75468,75506,76635,77411,81531,82362,82606,83118,83863,84975,...,11001115,11001901,11001910,11005004,11005829,11006412,11007160,11008686,11011328,11011787,11011845,11012714,11012984,11016498,11021278,11022077,11023034,11028540,11029920,11031016,11031213,11032664,11033381,11034407,11038926,11039510,11040046,11041479,11042580,11042860,11046434,11046547,11049819,11051185,11052263,11058766,11066701,11070317,11078368,11079929
TXN_DESC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
(JOO)GIMCHISEUKOOL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
(JOO)SINSEGYEOIMYEONSEJEOINCH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
*TWD-ANYTIMEFIT-EZYPAY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24pmnt.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42RACE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7-ELEVEN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
???????-????,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,11,0,0,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABERCROMBIE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# 用來方便印出使用者資料
def print_txn(n):
    print(df.iloc[n].name)

In [14]:
for i in range(5):
  print_txn(i)
  print()

(JOO)GIMCHISEUKOOL

(JOO)SINSEGYEOIMYEONSEJEOINCH

*TWD-ANYTIMEFIT-EZYPAY

-

24pmnt.com



In [15]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=100)
matrix = SVD.fit_transform(df)
matrix.shape

(5869, 100)

In [16]:
# sklearn NN
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='correlation', algorithm='brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='correlation',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
def find_related(name):
    i = df.index.get_loc(name)
    distances, indices = model_knn.kneighbors(matrix[i:i+1], n_neighbors=6)
    for i, d in zip(indices[0][1:], distances[0][1:]):
        print(df.iloc[i].name, d)

In [18]:
find_related('台中新光影城')

遠東百貨股份有限公司台中分公 0.10780291049839041
廣三ＳＯＧＯ 0.112616205699964
新光三越ＳＰ台中中港 0.1353555699468021
貳樓洋食有限公司台中公益分公 0.1415980013617324
勤美誠品綠園道 0.14448316722091703


In [19]:
find_related('友邦人壽')

遠雄人壽保險事業股份 0.2286346872027346
宅急網股份有限公司 0.24848041508834806
新光三越百貨桃園站前店 0.24992700854265293
台灣小米通訊有限公司 0.25540492668215353
全球人壽 0.25546739064095425


In [20]:
find_related('AGODA')

昇恒昌 0.09607502480953689
易遊網旅行社股份有限公司 0.10585519294536638
長榮航空股份有限公司 0.1168854552414359
ＩＫＥＡ宜家家居 0.13118489356805374
人杰老四川餐飲管理顧問股份有 0.14238816358770656


In [21]:
find_related('一卡通自動加值')

全聯鹽埔分公司 0.022326108911252307
統一精工華亞自助加油站 0.027231922485513538
震旦通訊 0.03440542139934388
坤業加油站有限公司景福營業所 0.061314652767223365
頂好Ｗｅｌｌｃｏｍｅ淡海店 0.06545916035540056


In [22]:
find_related('中友百貨公司')

中友百貨股份有限公司 0.0806554090613093
宮廷餐廳有限公司 0.12393943475477154
中友百貨股份有限公司台中分公 0.14164676336525683
新光三越百貨股份有限公司台中 0.15023697352257048
中友百貨宮廷餐廳 0.1692402879593531
